In [8]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores.in_memory import InMemoryVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
import numpy as np
import hashlib
import os
import re

# загружаем документы
def load_doc(listpath):
    docs = []
    for path in listpath:
        name, ext = os.path.splitext(path)
        if ext == '.pdf':
            loader = PyPDFLoader(
                file_path=path,
                mode="page",  # "page" - по страницам (по умолчанию), "single" - весь документ
                extract_images=False,  # извлекать изображения
            )
        elif ext == '.docx':
            loader = Docx2txtLoader(path)
        else:
            print('Формат не поддерживается.')
            continue


        doc = loader.load()
        docs.append(doc)

    return docs

# удаляем текст по шаблонам, а также перетягиваем страницы на уровень вверх, они становятся как документы.
def normaliz_cleanir_text(docs):
    new_docs = []
    for doc in docs:
        for page in doc:
            text = page.page_content
            text = text.lower()
            text = re.sub(r"^[]\d+\s*", "", text, flags=re.MULTILINE)   # удаление номеров страниц.
            text = re.sub(r"\s+", " ", text)   # свертка пробелов
            text = re.sub(r"(\w)\s(\w)-\n?", r"\1\2", text)   # удаление переносов, где есть пробел (видимо особенность пдф)
            text = re.sub(r"(\w)-\n?", r"\1", text)   # удаление переносов обычного
            text = re.sub(r"\n", " ", text)
            text = re.sub(r"[…]+", "", text)   # удаление группы точек в содержание
            d = text.encode('utf-8', 'ignore').decode('utf-8').strip()
            if page.metadata.get('page') is None:
               page.metadata['page'] = 1
            n_d = Document(page_content=d, metadata=page.metadata)
            new_docs.append(n_d)
    return new_docs

# делим куски на еще более мелкие чанки
def get_chunks(docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
    splitted_docs = text_splitter.split_documents(docs)
    return splitted_docs

# техническая функция вычисления похожести двух векторов
def cosine_similarity(v1: np.ndarray, v2: np.ndarray) -> float:
    return float(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))

# удаление дублей по хешу и по эмбеддингам. также удаление коротких кусков
def filter_and_dedup(docs, embedding_model, min_length=30, threshold_embedding=0.95):
    unique_hashes = set()
    embeddings = []

    filtered = []
    stats = {'duplicates_hash': 0, 'too_short': 0, 'duplicates_embedding': 0}
    for doc in docs:
        text = doc.page_content.strip()
        # короткие удаляем
        if len(text) < min_length:
            stats['too_short'] += 1
            continue

        # быстрый способ на равенство строк. чем длиннее строки тем быстрее хешем
        h = hashlib.md5(text.encode('utf-8')).hexdigest()
        if h in unique_hashes:
            stats['duplicates_hash'] += 1
            continue

        # получаем эмбеддинги и сравниваем на похожесть
        emb = embedding_model.embed_documents([text])[0]

        if len(embeddings) > 0:
            sims = [cosine_similarity(np.array(emb), np.array(e)) for e in embeddings]
            max_sim = max(sims)
            if max_sim < threshold_embedding:
                embeddings.append(emb)
            else:
                stats['duplicates_embedding'] += 1
                continue
        else:
            embeddings.append(emb)


        unique_hashes.add(h)
        filtered.append(doc)

    print(f"Первоначально: {len(docs)} чанков")
    print(f"Удалено дубликатов хеш: {stats['duplicates_hash']}, слишком коротких: \
               {stats['too_short']}, Удалено дубликатов эмбеддинги: {stats['duplicates_embedding']}")
    print(f"Осталось: {len(filtered)} чанков")
    return filtered

In [9]:
# специально указал один дубликат.
embed_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
path_doc=["docs/Теневые сады.pdf", 'docs/book.docx', "docs/Теневые сады.pdf"]

raw_docs = load_doc(path_doc)
clean_docs = normaliz_cleanir_text(raw_docs)
chunks = get_chunks(clean_docs)
good_docs = filter_and_dedup(chunks, embed_model, min_length=40, threshold_embedding=0.95)

print('-'*50)
print('Всего документов:', len(raw_docs), 'Всего страниц в документах:', len(clean_docs),
      'Всего чанков:', len(chunks), 'Всего чанков после удаления дубликатов:', len(good_docs), sep='\n')

Первоначально: 1626 чанков
Удалено дубликатов хеш: 452, слишком коротких:                6, Удалено дубликатов эмбеддинги: 6
Осталось: 1162 чанков
--------------------------------------------------
Всего документов:
3
Всего страниц в документах:
287
Всего чанков:
1626
Всего чанков после удаления дубликатов:
1162


In [10]:
for doc in good_docs:
    print(doc.page_content[:50], doc.metadata['page'],)

национальный исследовательский томский государстве 0
рас смотрено и утверждено методической комиссией б 1
дом томского университета, 2019. – 144 с. учебноме 1
для озеленения в условиях томской области пред лож 1
«ландшафт ная архитектура». издание рассчитано на  1
3 содержание предисловие.... 4 1. характеристика т 2
весной и в начале лета. 21 3.2. декоративные в сер 2
дополнительных источников литературы.. 130 указате 2
4 предисловие в данном пособии обобщен опыт обустр 3
растений флоры сибири в ландшафтном дизайне. предс 3
растений на теневых участках; основные виды многол 3
виды расте ний природной флоры сибири для различны 3
приемами выращивания видов растений природ ной фло 3
ямбуровым). наряду с общими рекомендациями предлож 3
5 растения разделены на 4 группы по периодам, в ко 4
размещения в теневых садах. значительно облегчает  4
контрольные вопросы, которые помогут системати зир 4
дисциплин ам «цветоводство», «рекреационное лесопо 4
6 1. характеристика теневого сада на любой озе